# Backtest Execution Workflow

**Systematic Macro Credit Research — Step 4 of 5**

This notebook executes individual signal backtests across user-selected strategies. It represents the fourth step in the systematic research workflow.

## Workflow Position

```
1. Data Download (01_data_download.ipynb)
   ↓
2. Signal Computation (02_signal_computation.ipynb)
   ↓
3. Signal Suitability Evaluation (03_suitability_evaluation.ipynb)
   ↓
4. Backtest Execution ← YOU ARE HERE
   ↓
5. Performance Analysis (05_analysis.ipynb)
```

## Prerequisites

**This notebook only loads cached data. It does not generate any new data.**

Required files from previous steps:
- **Signals:** `data/processed/signals.parquet` (from Step 2)
- **CDX spreads:** `data/cache/bloomberg/cdx_ig_5y.parquet` (from Step 1)

If you don't have Bloomberg Terminal access, run `generate_synthetic_data.py` before Step 2 to create the required cache files.

## What This Notebook Does

1. **Load Signals** — Read computed signals from Step 2
2. **Configure Selection** — Choose signals and strategies to backtest (defaults to all)
3. **Load CDX Data** — Read spreads from cache for P&L calculation
4. **Execute Backtest Matrix** — Run backtests for all signal-strategy pairs
5. **Compute Metrics** — Calculate performance statistics for each run
6. **Display Results** — Show comparison tables organized by signal
7. **Suggest Focus Pairs** — Auto-identify top performers for visualization
8. **Visualize Focus Pairs** — Generate equity curves, signals, and drawdowns
9. **Persist Results** — Save unified backtest results and metadata

## Outputs

- **Backtest Results:** `data/processed/backtest_results.parquet` (MultiIndex: signal, strategy)
- **Execution Metadata:** `logs/backtest_metadata.json`

## Key Design Patterns

- **Cache-Only:** Loads all data from previous workflow steps (no data generation)
- **User Control:** Manual signal/strategy selection independent of suitability results
- **Full Matrix:** Execute all selected combinations by default
- **Focus Visualization:** Plot only top performers to keep notebook manageable
- **Readable Tables:** Separate comparison tables per signal with strategies as columns

---

## 1. Imports and Configuration

Import dependencies and verify data availability.

In [29]:
import logging
from datetime import datetime
from pathlib import Path

import pandas as pd

from aponyx.config import DATA_DIR, LOGS_DIR, REGISTRY_PATH, STRATEGY_CATALOG_PATH
from aponyx.data.registry import DataRegistry
from aponyx.backtest import run_backtest
from aponyx.backtest.registry import StrategyRegistry
from aponyx.backtest.metrics import compute_performance_metrics
from aponyx.persistence import load_parquet, save_parquet, save_json
from aponyx.visualization import plot_equity_curve, plot_signal, plot_drawdown

# Configure logging for notebook
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
)
logger = logging.getLogger(__name__)

print("=" * 80)
print("BACKTEST EXECUTION WORKFLOW — Step 4 of 5")
print("=" * 80)
print(f"\nConfiguration:")
print(f"  Data directory: {DATA_DIR}")
print(f"  Logs directory: {LOGS_DIR}")
print(f"  Strategy catalog: {STRATEGY_CATALOG_PATH}")
print(f"\n✓ Imports complete")

BACKTEST EXECUTION WORKFLOW — Step 4 of 5

Configuration:
  Data directory: C:\Users\ROG3003\PythonProjects\aponyx\data
  Logs directory: C:\Users\ROG3003\PythonProjects\aponyx\logs
  Strategy catalog: C:\Users\ROG3003\PythonProjects\aponyx\src\aponyx\backtest\strategy_catalog.json

✓ Imports complete


## 2. Load Signals

Load computed signals from Step 2 and display available signals.

In [30]:
# Load signals
signals_path = DATA_DIR / "processed" / "signals.parquet"

if not signals_path.exists():
    raise FileNotFoundError(
        f"Signals file not found: {signals_path}\n"
        "Please run 02_signal_computation.ipynb first."
    )

signals = load_parquet(signals_path)

print(f"\n{'='*80}")
print(f"SIGNALS LOADED")
print(f"{'='*80}\n")
print(f"Path: {signals_path}")
print(f"Date range: {signals.index.min()} to {signals.index.max()}")
print(f"Observations: {len(signals):,}")
print(f"\nAvailable signals ({len(signals.columns)}):")
for col in signals.columns:
    print(f"  - {col}")
    
print(f"\nSignal statistics:")
print(signals.describe().to_markdown())

2025-11-09 13:32:31,926 - aponyx.persistence.parquet_io - INFO - Loading Parquet file: path=C:\Users\ROG3003\PythonProjects\aponyx\data\processed\signals.parquet, columns=all
2025-11-09 13:32:31,926 - aponyx.persistence.parquet_io - INFO - Loaded 1305 rows, 3 columns from C:\Users\ROG3003\PythonProjects\aponyx\data\processed\signals.parquet
2025-11-09 13:32:31,926 - aponyx.persistence.parquet_io - INFO - Loaded 1305 rows, 3 columns from C:\Users\ROG3003\PythonProjects\aponyx\data\processed\signals.parquet



SIGNALS LOADED

Path: C:\Users\ROG3003\PythonProjects\aponyx\data\processed\signals.parquet
Date range: 2020-11-09 00:00:00 to 2024-06-05 00:00:00
Observations: 1,305

Available signals (3):
  - cdx_etf_basis
  - cdx_vix_gap
  - spread_momentum

Signal statistics:
|       |   cdx_etf_basis |   cdx_vix_gap |   spread_momentum |
|:------|----------------:|--------------:|------------------:|
| count |    1296         |   1287        |      1285         |
| mean  |      -0.134172  |      0.028319 |        -0.0328406 |
| std   |       1.28824   |      1.20548  |         1.94327   |
| min   |      -3.32572   |     -3.79535  |        -5.71146   |
| 25%   |      -1.20386   |     -0.888449 |        -1.56492   |
| 50%   |      -0.0278169 |      0.104621 |        -0.0537993 |
| 75%   |       0.907069  |      0.914234 |         1.48435   |
| max   |       2.6169    |      3.36197  |         5.09683   |


## 3. Configure Signal Selection

Choose which signals to backtest. Default is all available signals.

**To narrow selection:** Edit the list below to include only desired signals.

In [31]:
# Signal selection: default to all available signals
# To narrow: selected_signals = ['cdx_vix_gap', 'spread_momentum']
selected_signals = list(signals.columns)

print(f"\n{'='*80}")
print(f"SIGNAL SELECTION")
print(f"{'='*80}\n")
print(f"Selected signals ({len(selected_signals)}):")
for signal in selected_signals:
    print(f"  ✓ {signal}")
    
# Validate selection
invalid_signals = [s for s in selected_signals if s not in signals.columns]
if invalid_signals:
    raise ValueError(f"Invalid signal names: {invalid_signals}")


SIGNAL SELECTION

Selected signals (3):
  ✓ cdx_etf_basis
  ✓ cdx_vix_gap
  ✓ spread_momentum


## 4. Load Strategy Catalog

Load strategy definitions and display available strategies.

In [32]:
# Load strategy registry
strategy_registry = StrategyRegistry(STRATEGY_CATALOG_PATH)
all_strategies = strategy_registry.list_all()

print(f"\n{'='*80}")
print(f"STRATEGY CATALOG")
print(f"{'='*80}\n")
print(f"Catalog path: {STRATEGY_CATALOG_PATH}")
print(f"Total strategies: {len(all_strategies)}\n")

# Display strategy catalog as table
strategy_data = []
for name, metadata in all_strategies.items():
    strategy_data.append({
        "Strategy": name,
        "Entry σ": f"{metadata.entry_threshold:.2f}",
        "Exit σ": f"{metadata.exit_threshold:.2f}",
        "Enabled": "✓" if metadata.enabled else "✗",
        "Description": metadata.description,
    })

strategy_table = pd.DataFrame(strategy_data)
print(strategy_table.to_markdown(index=False))

2025-11-09 13:32:31,952 - aponyx.backtest.registry - INFO - Loaded strategy registry: catalog=C:\Users\ROG3003\PythonProjects\aponyx\src\aponyx\backtest\strategy_catalog.json, strategies=4, enabled=3



STRATEGY CATALOG

Catalog path: C:\Users\ROG3003\PythonProjects\aponyx\src\aponyx\backtest\strategy_catalog.json
Total strategies: 4

| Strategy     |   Entry σ |   Exit σ | Enabled   | Description                                                      |
|:-------------|----------:|---------:|:----------|:-----------------------------------------------------------------|
| conservative |      2    |     1    | ✓         | Conservative thresholds for low-turnover, high-conviction trades |
| balanced     |      1.5  |     0.75 | ✓         | Balanced thresholds for moderate turnover and signal filtering   |
| aggressive   |      1    |     0.5  | ✓         | Aggressive thresholds for high-turnover, responsive trading      |
| experimental |      0.75 |     0.25 | ✗         | Experimental thresholds for research and testing                 |


## 5. Configure Strategy Selection

Choose which strategies to test. Default is all strategies (including experimental).

**To narrow selection:** Edit the list below to include only desired strategies.

In [33]:
# Strategy selection: default to all strategies
# To narrow: selected_strategies = ['conservative', 'balanced', 'aggressive']
selected_strategies = list(all_strategies.keys())

print(f"\n{'='*80}")
print(f"STRATEGY SELECTION")
print(f"{'='*80}\n")
print(f"Selected strategies ({len(selected_strategies)}):")
for strategy in selected_strategies:
    metadata = all_strategies[strategy]
    print(f"  ✓ {strategy:15s} (Entry: {metadata.entry_threshold:.2f}σ, Exit: {metadata.exit_threshold:.2f}σ)")
    
# Validate selection
invalid_strategies = [s for s in selected_strategies if s not in all_strategies]
if invalid_strategies:
    raise ValueError(f"Invalid strategy names: {invalid_strategies}")
    
print(f"\nBacktest matrix: {len(selected_signals)} signals × {len(selected_strategies)} strategies = {len(selected_signals) * len(selected_strategies)} runs")


STRATEGY SELECTION

Selected strategies (4):
  ✓ conservative    (Entry: 2.00σ, Exit: 1.00σ)
  ✓ balanced        (Entry: 1.50σ, Exit: 0.75σ)
  ✓ aggressive      (Entry: 1.00σ, Exit: 0.50σ)
  ✓ experimental    (Entry: 0.75σ, Exit: 0.25σ)

Backtest matrix: 3 signals × 4 strategies = 12 runs


## 6. Load CDX Spread Data

Load CDX spreads from cache for P&L calculation.

In [34]:
# Load CDX data from cache using registry
registry = DataRegistry(REGISTRY_PATH, DATA_DIR)

print(f"\n{'='*80}")
print(f"LOADING CDX DATA FROM CACHE")
print(f"{'='*80}\n")

# Find CDX IG 5Y in registry
cdx_datasets = [
    name for name in registry.list_datasets(instrument="cdx")
    if "cdx_ig_5y" in registry.get_dataset_info(name).get("metadata", {}).get("params", {}).get("security", "")
]

if not cdx_datasets:
    raise FileNotFoundError(
        "CDX IG 5Y data not found in registry.\n"
        "Please run 01_data_download.ipynb first."
    )

# Use the first matching dataset
cdx_dataset_name = cdx_datasets[0]
cdx_info = registry.get_dataset_info(cdx_dataset_name)
cdx_cache_path = Path(cdx_info["file_path"])

print(f"Registry entry: {cdx_dataset_name}")
print(f"Cache path: {cdx_cache_path}")

cdx_df = load_parquet(cdx_cache_path)
print(f"✓ Loaded cached CDX data: {len(cdx_df):,} observations")

# Align CDX data to signal dates
cdx_spread = cdx_df['spread'].reindex(signals.index).ffill()

print(f"\nCDX spread statistics:")
print(f"  Mean: {cdx_spread.mean():.2f} bps")
print(f"  Std: {cdx_spread.std():.2f} bps")
print(f"  Min: {cdx_spread.min():.2f} bps")
print(f"  Max: {cdx_spread.max():.2f} bps")
print(f"  Missing: {cdx_spread.isna().sum()} observations")

2025-11-09 13:32:31,973 - aponyx.persistence.json_io - INFO - Loading JSON from C:\Users\ROG3003\PythonProjects\aponyx\data\registry.json
2025-11-09 13:32:31,974 - aponyx.data.registry - INFO - Loaded existing registry: path=C:\Users\ROG3003\PythonProjects\aponyx\data\registry.json, datasets=3
2025-11-09 13:32:31,975 - aponyx.persistence.parquet_io - INFO - Loading Parquet file: path=C:\Users\ROG3003\PythonProjects\aponyx\data\cache\file\cdx_c3bedc49b771b0f2.parquet, columns=all
2025-11-09 13:32:31,978 - aponyx.persistence.parquet_io - INFO - Loaded 1305 rows, 2 columns from C:\Users\ROG3003\PythonProjects\aponyx\data\cache\file\cdx_c3bedc49b771b0f2.parquet



LOADING CDX DATA FROM CACHE

Registry entry: cache_cdx_c3bedc49b771b0f2
Cache path: C:\Users\ROG3003\PythonProjects\aponyx\data\cache\file\cdx_c3bedc49b771b0f2.parquet
✓ Loaded cached CDX data: 1,305 observations

CDX spread statistics:
  Mean: 58.79 bps
  Std: 11.18 bps
  Min: 21.65 bps
  Max: 89.43 bps
  Missing: 0 observations


## 7. Execute Backtest Matrix

Run backtests for all selected signal-strategy pairs.

In [35]:
print(f"\n{'='*80}")
print(f"EXECUTING BACKTEST MATRIX")
print(f"{'='*80}\n")

# Store results
backtest_results = {}
execution_start = datetime.now()

# Execute backtests
total_runs = len(selected_signals) * len(selected_strategies)
current_run = 0

for signal_name in selected_signals:
    for strategy_name in selected_strategies:
        current_run += 1
        print(f"[{current_run}/{total_runs}] Running: {signal_name} × {strategy_name}")
        
        # Get strategy configuration
        strategy_metadata = all_strategies[strategy_name]
        config = strategy_metadata.to_config(
            position_size=10.0,  # $10MM notional
            transaction_cost_bps=1.0,  # 1bp round-trip
            max_holding_days=None,  # No time limit
            dv01_per_million=4750.0,  # CDX IG 5Y typical DV01
        )
        
        # Run backtest
        signal = signals[signal_name]
        result = run_backtest(
            composite_signal=signal,
            spread=cdx_spread,
            config=config,
        )
        
        # Store with composite key
        backtest_results[(signal_name, strategy_name)] = result

execution_end = datetime.now()
execution_time = (execution_end - execution_start).total_seconds()

print(f"\n✓ Backtest matrix complete: {total_runs} runs in {execution_time:.1f}s")
print(f"  Average: {execution_time/total_runs:.2f}s per run")

2025-11-09 13:32:31,988 - aponyx.backtest.engine - INFO - Starting backtest: dates=1305, entry_threshold=2.00, position_size=10.0MM
2025-11-09 13:32:32,024 - aponyx.backtest.engine - INFO - Backtest complete: trades=53, total_pnl=$-12023174, avg_per_trade=$-226852
2025-11-09 13:32:32,025 - aponyx.backtest.engine - INFO - Starting backtest: dates=1305, entry_threshold=1.50, position_size=10.0MM
2025-11-09 13:32:32,024 - aponyx.backtest.engine - INFO - Backtest complete: trades=53, total_pnl=$-12023174, avg_per_trade=$-226852
2025-11-09 13:32:32,025 - aponyx.backtest.engine - INFO - Starting backtest: dates=1305, entry_threshold=1.50, position_size=10.0MM
2025-11-09 13:32:32,058 - aponyx.backtest.engine - INFO - Backtest complete: trades=87, total_pnl=$-10529419, avg_per_trade=$-121028
2025-11-09 13:32:32,060 - aponyx.backtest.engine - INFO - Starting backtest: dates=1305, entry_threshold=1.00, position_size=10.0MM
2025-11-09 13:32:32,058 - aponyx.backtest.engine - INFO - Backtest comple


EXECUTING BACKTEST MATRIX

[1/12] Running: cdx_etf_basis × conservative
[2/12] Running: cdx_etf_basis × balanced
[3/12] Running: cdx_etf_basis × aggressive
[4/12] Running: cdx_etf_basis × experimental


2025-11-09 13:32:32,130 - aponyx.backtest.engine - INFO - Backtest complete: trades=87, total_pnl=$67535989, avg_per_trade=$776276
2025-11-09 13:32:32,131 - aponyx.backtest.engine - INFO - Starting backtest: dates=1305, entry_threshold=2.00, position_size=10.0MM
2025-11-09 13:32:32,131 - aponyx.backtest.engine - INFO - Starting backtest: dates=1305, entry_threshold=2.00, position_size=10.0MM


[5/12] Running: cdx_vix_gap × conservative


2025-11-09 13:32:32,164 - aponyx.backtest.engine - INFO - Backtest complete: trades=48, total_pnl=$11030752, avg_per_trade=$229807
2025-11-09 13:32:32,164 - aponyx.backtest.engine - INFO - Starting backtest: dates=1305, entry_threshold=1.50, position_size=10.0MM
2025-11-09 13:32:32,164 - aponyx.backtest.engine - INFO - Starting backtest: dates=1305, entry_threshold=1.50, position_size=10.0MM
2025-11-09 13:32:32,199 - aponyx.backtest.engine - INFO - Backtest complete: trades=87, total_pnl=$-51766580, avg_per_trade=$-595018
2025-11-09 13:32:32,200 - aponyx.backtest.engine - INFO - Starting backtest: dates=1305, entry_threshold=1.00, position_size=10.0MM
2025-11-09 13:32:32,199 - aponyx.backtest.engine - INFO - Backtest complete: trades=87, total_pnl=$-51766580, avg_per_trade=$-595018
2025-11-09 13:32:32,200 - aponyx.backtest.engine - INFO - Starting backtest: dates=1305, entry_threshold=1.00, position_size=10.0MM
2025-11-09 13:32:32,232 - aponyx.backtest.engine - INFO - Backtest complete

[6/12] Running: cdx_vix_gap × balanced
[7/12] Running: cdx_vix_gap × aggressive
[8/12] Running: cdx_vix_gap × experimental
[9/12] Running: spread_momentum × conservative


2025-11-09 13:32:32,304 - aponyx.backtest.engine - INFO - Starting backtest: dates=1305, entry_threshold=1.50, position_size=10.0MM
2025-11-09 13:32:32,338 - aponyx.backtest.engine - INFO - Backtest complete: trades=105, total_pnl=$-50075038, avg_per_trade=$-476905
2025-11-09 13:32:32,340 - aponyx.backtest.engine - INFO - Starting backtest: dates=1305, entry_threshold=1.00, position_size=10.0MM
2025-11-09 13:32:32,338 - aponyx.backtest.engine - INFO - Backtest complete: trades=105, total_pnl=$-50075038, avg_per_trade=$-476905
2025-11-09 13:32:32,340 - aponyx.backtest.engine - INFO - Starting backtest: dates=1305, entry_threshold=1.00, position_size=10.0MM


[10/12] Running: spread_momentum × balanced
[11/12] Running: spread_momentum × aggressive


2025-11-09 13:32:32,374 - aponyx.backtest.engine - INFO - Backtest complete: trades=112, total_pnl=$-7485296, avg_per_trade=$-66833
2025-11-09 13:32:32,374 - aponyx.backtest.engine - INFO - Starting backtest: dates=1305, entry_threshold=0.75, position_size=10.0MM
2025-11-09 13:32:32,374 - aponyx.backtest.engine - INFO - Starting backtest: dates=1305, entry_threshold=0.75, position_size=10.0MM
2025-11-09 13:32:32,408 - aponyx.backtest.engine - INFO - Backtest complete: trades=84, total_pnl=$-112395046, avg_per_trade=$-1338036


[12/12] Running: spread_momentum × experimental

✓ Backtest matrix complete: 12 runs in 0.4s
  Average: 0.04s per run


## 8. Compute Performance Metrics

Calculate performance statistics for all backtest runs.

In [36]:
print(f"\n{'='*80}")
print(f"COMPUTING PERFORMANCE METRICS")
print(f"{'='*80}\n")

# Compute metrics for all runs
metrics_data = []

for (signal_name, strategy_name), result in backtest_results.items():
    metrics = compute_performance_metrics(result.pnl, result.positions)
    
    metrics_data.append({
        "signal": signal_name,
        "strategy": strategy_name,
        "sharpe": metrics.sharpe_ratio,
        "sortino": metrics.sortino_ratio,
        "max_dd": metrics.max_drawdown,
        "calmar": metrics.calmar_ratio,
        "total_pnl": metrics.total_return,
        "ann_return": metrics.annualized_return,
        "ann_vol": metrics.annualized_volatility,
        "hit_rate": metrics.hit_rate,
        "avg_win": metrics.avg_win,
        "avg_loss": metrics.avg_loss,
        "win_loss": metrics.win_loss_ratio,
        "n_trades": metrics.n_trades,
        "avg_days": metrics.avg_holding_days,
    })

metrics_df = pd.DataFrame(metrics_data)
print(f"✓ Computed metrics for {len(metrics_df)} backtest runs")

2025-11-09 13:32:32,415 - aponyx.backtest.metrics - INFO - Computing performance metrics
2025-11-09 13:32:32,420 - aponyx.backtest.metrics - INFO - Metrics computed: sharpe=-0.48, max_dd=$-43049117, hit_rate=37.7%
2025-11-09 13:32:32,420 - aponyx.backtest.metrics - INFO - Computing performance metrics
2025-11-09 13:32:32,425 - aponyx.backtest.metrics - INFO - Metrics computed: sharpe=-0.34, max_dd=$-40097162, hit_rate=33.3%
2025-11-09 13:32:32,420 - aponyx.backtest.metrics - INFO - Metrics computed: sharpe=-0.48, max_dd=$-43049117, hit_rate=37.7%
2025-11-09 13:32:32,420 - aponyx.backtest.metrics - INFO - Computing performance metrics
2025-11-09 13:32:32,425 - aponyx.backtest.metrics - INFO - Metrics computed: sharpe=-0.34, max_dd=$-40097162, hit_rate=33.3%
2025-11-09 13:32:32,425 - aponyx.backtest.metrics - INFO - Computing performance metrics
2025-11-09 13:32:32,429 - aponyx.backtest.metrics - INFO - Metrics computed: sharpe=-1.32, max_dd=$-61295928, hit_rate=40.9%
2025-11-09 13:32:32


COMPUTING PERFORMANCE METRICS

✓ Computed metrics for 12 backtest runs


## 9. Display Performance Tables

Show comparison tables organized by signal with strategies as columns.

In [37]:
print(f"\n{'='*80}")
print(f"PERFORMANCE COMPARISON BY SIGNAL")
print(f"{'='*80}\n")

# Create separate tables for each signal
for signal_name in selected_signals:
    signal_metrics = metrics_df[metrics_df['signal'] == signal_name].copy()
    
    # Pivot to have strategies as columns
    comparison_data = []
    metric_labels = [
        ("Sharpe Ratio", "sharpe", "{:.2f}"),
        ("Sortino Ratio", "sortino", "{:.2f}"),
        ("Calmar Ratio", "calmar", "{:.2f}"),
        ("Total P&L ($)", "total_pnl", "${:,.0f}"),
        ("Ann. Return ($)", "ann_return", "${:,.0f}"),
        ("Ann. Vol ($)", "ann_vol", "${:,.0f}"),
        ("Max DD ($)", "max_dd", "${:,.0f}"),
        ("Hit Rate (%)", "hit_rate", "{:.1%}"),
        ("Avg Win ($)", "avg_win", "${:,.0f}"),
        ("Avg Loss ($)", "avg_loss", "${:,.0f}"),
        ("Win/Loss Ratio", "win_loss", "{:.2f}"),
        ("# Trades", "n_trades", "{:.0f}"),
        ("Avg Days Held", "avg_days", "{:.1f}"),
    ]
    
    for label, col, fmt in metric_labels:
        row = {"Metric": label}
        for _, row_data in signal_metrics.iterrows():
            strategy = row_data['strategy']
            value = row_data[col]
            row[strategy] = fmt.format(value)
        comparison_data.append(row)
    
    comparison_df = pd.DataFrame(comparison_data)
    
    print(f"\n{'─'*80}")
    print(f"Signal: {signal_name}")
    print(f"{'─'*80}\n")
    print(comparison_df.to_markdown(index=False))
    print()


PERFORMANCE COMPARISON BY SIGNAL


────────────────────────────────────────────────────────────────────────────────
Signal: cdx_etf_basis
────────────────────────────────────────────────────────────────────────────────

| Metric          | conservative   | balanced     | aggressive   | experimental   |
|:----------------|:---------------|:-------------|:-------------|:---------------|
| Sharpe Ratio    | -0.48          | -0.34        | -1.32        | 1.45           |
| Sortino Ratio   | -0.34          | -0.35        | -1.49        | 2.10           |
| Calmar Ratio    | -0.05          | -0.05        | -0.14        | 0.19           |
| Total P&L ($)   | $-12,023,174   | $-10,529,419 | $-45,076,786 | $67,535,989    |
| Ann. Return ($) | $-2,337,839    | $-2,047,387  | $-8,764,931  | $13,131,998    |
| Ann. Vol ($)    | $4,919,532     | $6,065,017   | $6,654,858   | $9,061,182     |
| Max DD ($)      | $-43,049,117   | $-40,097,162 | $-61,295,928 | $-67,436,416   |
| Hit Rate (%)    | 37.

## 10. Suggest Focus Pairs for Visualization

Auto-identify top-performing signal-strategy pairs based on Sharpe ratio.

In [38]:
print(f"\n{'='*80}")
print(f"TOP PERFORMERS (by Sharpe Ratio)")
print(f"{'='*80}\n")

# Sort by Sharpe ratio and display top performers
top_performers = metrics_df.nlargest(10, 'sharpe')[['signal', 'strategy', 'sharpe', 'total_pnl', 'max_dd', 'n_trades']]
top_performers_display = top_performers.copy()
top_performers_display.columns = ['Signal', 'Strategy', 'Sharpe', 'Total P&L', 'Max DD', 'Trades']
top_performers_display['Total P&L'] = top_performers_display['Total P&L'].apply(lambda x: f"${x:,.0f}")
top_performers_display['Max DD'] = top_performers_display['Max DD'].apply(lambda x: f"${x:,.0f}")
top_performers_display['Sharpe'] = top_performers_display['Sharpe'].apply(lambda x: f"{x:.2f}")

print(top_performers_display.to_markdown(index=False))

# Auto-suggest top 3 as focus pairs
suggested_focus = [(row['signal'], row['strategy']) for _, row in top_performers.head(3).iterrows()]

print(f"\n{'─'*80}")
print(f"Suggested focus pairs for visualization (top 3 by Sharpe):")
print(f"{'─'*80}\n")
for i, (signal, strategy) in enumerate(suggested_focus, 1):
    sharpe = metrics_df[(metrics_df['signal'] == signal) & (metrics_df['strategy'] == strategy)]['sharpe'].iloc[0]
    print(f"  {i}. {signal} × {strategy} (Sharpe: {sharpe:.2f})")

print(f"\nTo customize, edit focus_pairs list in next cell.")


TOP PERFORMERS (by Sharpe Ratio)

| Signal          | Strategy     |   Sharpe | Total P&L    | Max DD        |   Trades |
|:----------------|:-------------|---------:|:-------------|:--------------|---------:|
| cdx_etf_basis   | experimental |     1.45 | $67,535,989  | $-67,436,416  |       87 |
| cdx_vix_gap     | conservative |     1.07 | $11,030,752  | $-8,449,596   |       48 |
| spread_momentum | conservative |     0.66 | $17,218,624  | $-45,814,391  |      102 |
| spread_momentum | aggressive   |    -0.19 | $-7,485,296  | $-84,971,432  |      112 |
| cdx_etf_basis   | balanced     |    -0.34 | $-10,529,419 | $-40,097,162  |       87 |
| cdx_etf_basis   | conservative |    -0.48 | $-12,023,174 | $-43,049,117  |       53 |
| cdx_vix_gap     | experimental |    -1.16 | $-44,160,406 | $-73,570,316  |      103 |
| cdx_etf_basis   | aggressive   |    -1.32 | $-45,076,786 | $-61,295,928  |      110 |
| spread_momentum | balanced     |    -1.37 | $-50,075,038 | $-126,482,282 |      105

## 11. Configure Focus Pairs for Visualization

Select which signal-strategy pairs to visualize. Default is top 3 by Sharpe ratio.

**To customize:** Edit the list below to visualize different pairs.

In [39]:
# Focus pairs for visualization: default to top 3 by Sharpe
# To customize: focus_pairs = [('cdx_vix_gap', 'balanced'), ('spread_momentum', 'aggressive')]
focus_pairs = suggested_focus

print(f"\n{'='*80}")
print(f"FOCUS PAIRS FOR VISUALIZATION")
print(f"{'='*80}\n")
print(f"Selected {len(focus_pairs)} pairs for detailed visualization:")
for i, (signal, strategy) in enumerate(focus_pairs, 1):
    print(f"  {i}. {signal} × {strategy}")

# Validate focus pairs
for signal, strategy in focus_pairs:
    if signal not in selected_signals:
        raise ValueError(f"Focus pair signal '{signal}' not in selected signals")
    if strategy not in selected_strategies:
        raise ValueError(f"Focus pair strategy '{strategy}' not in selected strategies")


FOCUS PAIRS FOR VISUALIZATION

Selected 3 pairs for detailed visualization:
  1. cdx_etf_basis × experimental
  2. cdx_vix_gap × conservative
  3. spread_momentum × conservative


## 12. Visualize Focus Pairs — Equity Curves

Plot cumulative P&L with drawdown shading for each focus pair.

In [40]:
print(f"\n{'='*80}")
print(f"EQUITY CURVES — Focus Pairs")
print(f"{'='*80}\n")

for signal_name, strategy_name in focus_pairs:
    result = backtest_results[(signal_name, strategy_name)]
    metrics = metrics_df[
        (metrics_df['signal'] == signal_name) & 
        (metrics_df['strategy'] == strategy_name)
    ].iloc[0]
    
    title = f"Equity Curve: {signal_name} × {strategy_name} (Sharpe: {metrics['sharpe']:.2f})"
    fig = plot_equity_curve(
        result.pnl['net_pnl'],
        title=title,
        show_drawdown_shading=True,
    )
    fig.show()
    print(f"\n✓ Plotted: {signal_name} × {strategy_name}\n")

2025-11-09 13:32:32,524 - aponyx.visualization.plots - INFO - Plotting equity curve: 1296 observations



EQUITY CURVES — Focus Pairs



2025-11-09 13:32:32,966 - aponyx.visualization.plots - INFO - Plotting equity curve: 1287 observations



✓ Plotted: cdx_etf_basis × experimental



2025-11-09 13:32:33,282 - aponyx.visualization.plots - INFO - Plotting equity curve: 1285 observations



✓ Plotted: cdx_vix_gap × conservative




✓ Plotted: spread_momentum × conservative



## 13. Visualize Focus Pairs — Signal Overlays

Plot signal time series with strategy entry/exit thresholds.

In [41]:
print(f"\n{'='*80}")
print(f"SIGNAL OVERLAYS — Focus Pairs")
print(f"{'='*80}\n")

for signal_name, strategy_name in focus_pairs:
    strategy_metadata = all_strategies[strategy_name]
    signal = signals[signal_name]
    
    title = f"Signal: {signal_name} × {strategy_name} Strategy"
    
    # Plot with entry/exit thresholds
    threshold_lines = [
        strategy_metadata.entry_threshold,
        -strategy_metadata.entry_threshold,
        strategy_metadata.exit_threshold,
        -strategy_metadata.exit_threshold,
    ]
    
    fig = plot_signal(
        signal,
        title=title,
        threshold_lines=threshold_lines,
    )
    
    # Add annotations for thresholds
    fig.add_annotation(
        text=f"Entry: ±{strategy_metadata.entry_threshold:.2f}σ | Exit: ±{strategy_metadata.exit_threshold:.2f}σ",
        xref="paper", yref="paper",
        x=0.5, y=1.05,
        showarrow=False,
        font=dict(size=10, color="gray"),
    )
    
    fig.show()
    print(f"\n✓ Plotted: {signal_name} × {strategy_name}\n")

2025-11-09 13:32:33,406 - aponyx.visualization.plots - INFO - Plotting signal: 1305 observations



SIGNAL OVERLAYS — Focus Pairs



2025-11-09 13:32:33,467 - aponyx.visualization.plots - INFO - Plotting signal: 1305 observations



✓ Plotted: cdx_etf_basis × experimental



2025-11-09 13:32:33,517 - aponyx.visualization.plots - INFO - Plotting signal: 1305 observations



✓ Plotted: cdx_vix_gap × conservative




✓ Plotted: spread_momentum × conservative



## 14. Visualize Focus Pairs — Drawdowns

Plot underwater drawdown charts for each focus pair.

In [42]:
print(f"\n{'='*80}")
print(f"DRAWDOWN ANALYSIS — Focus Pairs")
print(f"{'='*80}\n")

for signal_name, strategy_name in focus_pairs:
    result = backtest_results[(signal_name, strategy_name)]
    metrics = metrics_df[
        (metrics_df['signal'] == signal_name) & 
        (metrics_df['strategy'] == strategy_name)
    ].iloc[0]
    
    title = f"Drawdown: {signal_name} × {strategy_name} (Max DD: ${metrics['max_dd']:,.0f})"
    fig = plot_drawdown(
        result.pnl['net_pnl'],
        title=title,
        show_underwater_chart=True,
    )
    fig.show()
    print(f"\n✓ Plotted: {signal_name} × {strategy_name}\n")

2025-11-09 13:32:33,601 - aponyx.visualization.plots - INFO - Plotting drawdown: 1296 observations



DRAWDOWN ANALYSIS — Focus Pairs



2025-11-09 13:32:33,638 - aponyx.visualization.plots - INFO - Plotting drawdown: 1287 observations



✓ Plotted: cdx_etf_basis × experimental



2025-11-09 13:32:33,691 - aponyx.visualization.plots - INFO - Plotting drawdown: 1285 observations



✓ Plotted: cdx_vix_gap × conservative




✓ Plotted: spread_momentum × conservative



## 15. Persist Backtest Results

Save unified backtest results and execution metadata.

In [43]:
print(f"\n{'='*80}")
print(f"PERSISTING BACKTEST RESULTS")
print(f"{'='*80}\n")

# Combine all results into single DataFrame with MultiIndex
all_positions = []
all_pnl = []

for (signal_name, strategy_name), result in backtest_results.items():
    # Add signal/strategy identifiers
    positions_with_id = result.positions.copy()
    positions_with_id['signal'] = signal_name
    positions_with_id['strategy'] = strategy_name
    
    pnl_with_id = result.pnl.copy()
    pnl_with_id['signal'] = signal_name
    pnl_with_id['strategy'] = strategy_name
    
    all_positions.append(positions_with_id)
    all_pnl.append(pnl_with_id)

# Combine and create MultiIndex
combined_positions = pd.concat(all_positions)
combined_pnl = pd.concat(all_pnl)

# Set MultiIndex: (signal, strategy, date)
combined_positions = combined_positions.reset_index()
combined_positions = combined_positions.set_index(['signal', 'strategy', 'date'])

combined_pnl = combined_pnl.reset_index()
combined_pnl = combined_pnl.set_index(['signal', 'strategy', 'date'])

# Save combined results
results_path = DATA_DIR / "processed" / "backtest_results.parquet"
save_parquet(combined_pnl, results_path)
print(f"✓ Saved backtest results: {results_path}")
print(f"  Shape: {combined_pnl.shape}")
print(f"  Pairs: {len(backtest_results)}")

# Save execution metadata
metadata = {
    "timestamp": datetime.now().isoformat(),
    "execution_time_seconds": execution_time,
    "configuration": {
        "selected_signals": selected_signals,
        "selected_strategies": selected_strategies,
        "focus_pairs": [(s, st) for s, st in focus_pairs],
        "position_size": 10.0,
        "transaction_cost_bps": 1.0,
        "dv01_per_million": 4750.0,
        "data_source": cdx_info.get("metadata", {}).get("cache_key", "unknown"),
    },
    "summary": {
        "total_runs": len(backtest_results),
        "date_range": {
            "start": str(signals.index.min()),
            "end": str(signals.index.max()),
        },
        "top_sharpe": {
            "signal": top_performers.iloc[0]['signal'],
            "strategy": top_performers.iloc[0]['strategy'],
            "value": float(top_performers.iloc[0]['sharpe']),
        },
    },
}

metadata_path = LOGS_DIR / "backtest_metadata.json"
save_json(metadata, metadata_path)
print(f"✓ Saved execution metadata: {metadata_path}")

2025-11-09 13:32:33,759 - aponyx.persistence.parquet_io - INFO - Saving DataFrame to Parquet: path=C:\Users\ROG3003\PythonProjects\aponyx\data\processed\backtest_results.parquet, rows=15472, columns=4, compression=snappy



PERSISTING BACKTEST RESULTS

✓ Saved backtest results: C:\Users\ROG3003\PythonProjects\aponyx\data\processed\backtest_results.parquet
  Shape: (15472, 4)
  Pairs: 12


2025-11-09 13:32:33,772 - aponyx.persistence.json_io - INFO - Saving JSON to C:\Users\ROG3003\PythonProjects\aponyx\logs\backtest_metadata.json (4 top-level keys)


✓ Saved execution metadata: C:\Users\ROG3003\PythonProjects\aponyx\logs\backtest_metadata.json


---

## Workflow Complete

Backtest execution successful! Individual signal backtests have been executed across all selected strategies and results are ready for analysis.

### What Was Accomplished

✓ **Signals Loaded** — Imported computed signals from Step 2  
✓ **Strategies Selected** — User-configured strategy subset (or all strategies)  
✓ **CDX Data Loaded** — Spreads loaded from cache via DataRegistry  
✓ **Backtest Matrix Executed** — All signal-strategy pairs tested  
✓ **Metrics Computed** — Performance statistics for each run  
✓ **Results Displayed** — Comparison tables by signal  
✓ **Focus Pairs Visualized** — Equity curves, signals, and drawdowns  
✓ **Outputs Persisted** — Unified results and metadata saved

### Data Flow

```
Signals (Step 2) + CDX Spreads (Step 1)
    ↓
Backtest Execution (this notebook)
├─ Strategy configuration from catalog
├─ Backtest matrix execution
└─ Performance metrics computation
    ↓
Backtest Results
├─ MultiIndex: (signal, strategy, date)
├─ P&L and positions for all runs
└─ Metadata with top performers
    ↓
Performance Analysis (Step 5)
```

### Re-Running This Notebook

- **Signal/Strategy Selection:** Edit cells 3 and 5 to narrow focus
- **Focus Pairs:** Edit cell 11 to customize visualizations
- **Backtest Rerun:** Results are recomputed from scratch each run
- **Outputs:** Overwrites `backtest_results.parquet` and `backtest_metadata.json`
- **Configuration:** Modify position size, costs, or DV01 in cell 7

### Key Files Generated

```
data/
└── processed/
    └── backtest_results.parquet (MultiIndex: signal, strategy, date)

logs/
└── backtest_metadata.json (execution summary and top performers)
```

### Troubleshooting

**Signals file not found:**
- Run `02_signal_computation.ipynb` first
- Verify file exists: `data/processed/signals.parquet`
- Check DATA_DIR configuration

**CDX data not found:**
- Run `01_data_download.ipynb` first
- Check registry contains CDX IG 5Y dataset
- Verify cache files exist in `data/cache/`

**Backtest execution errors:**
- Review ERROR logs for missing data or invalid configuration
- Check signal and CDX spread alignment (DatetimeIndex)
- Verify strategy catalog contains valid threshold definitions
- Ensure DV01 and position size are reasonable

**Poor performance results:**
- Review strategy thresholds (may be too aggressive or conservative)
- Check signal statistics (mean ~0, std ~1 for z-scores)
- Verify transaction costs are realistic (1bp is typical)
- Consider signal refinement or different strategy configurations

**Visualization issues:**
- Ensure plotly installed: `uv sync --extra viz`
- Check focus_pairs list contains valid signal-strategy combinations
- Verify matplotlib/plotly can render in Jupyter environment
- Reduce number of focus pairs if plots are too many